# Construction d'un modèle de survie par stacking

Ce notebook présente la construction d'un modèle de prédiction de survie utilisant une approche de stacking (empilement) combinant plusieurs algorithmes d'apprentissage automatique.

## Objectif

Prédire la survie de patients en utilisant des données cliniques et moléculaires, en combinant des modèles basés sur les arbres (Random Forest) et le boosting (XGBoost) via une approche de méta-apprentissage.

## 1. Chargement et préparation des données

Nous commençons par charger et fusionner les données cliniques et moléculaires via la clé ID. Les variables cliniques (centre, compte de blastes, numérations sanguines, profil cytogénétique) sont imputées par la médiane lorsque nécessaire, puis certaines d'entre elles sont transformées (p. ex. log sur WBC, ANC pour réduire l'asymétrie).

In [1]:
import pandas as pd
import numpy as np
import zipfile

# Chargement des données cliniques et moléculaires
clin = pd.read_csv('X_train/clinical_train.csv')
mol = pd.read_csv('X_train/molecular_train.csv')
target = pd.read_csv('target_train.csv')

print(f"Données cliniques: {clin.shape}")
print(f"Données moléculaires: {mol.shape}")
print(f"Données cibles: {target.shape}")

# Aperçu des données
print("\nAperçu des données cliniques:")
print(clin.head())
print("\nAperçu des données moléculaires:")
print(mol.head())
print("\nAperçu des données cibles:")
print(target.head())

Données cliniques: (3323, 9)
Données moléculaires: (10935, 11)
Données cibles: (3323, 3)

Aperçu des données cliniques:
        ID CENTER  BM_BLAST    WBC  ANC  MONOCYTES    HB    PLT  \
0  P132697    MSK      14.0    2.8  0.2        0.7   7.6  119.0   
1  P132698    MSK       1.0    7.4  2.4        0.1  11.6   42.0   
2  P116889    MSK      15.0    3.7  2.1        0.1  14.2   81.0   
3  P132699    MSK       1.0    3.9  1.9        0.1   8.9   77.0   
4  P132700    MSK       6.0  128.0  9.7        0.9  11.1  195.0   

                          CYTOGENETICS  
0      46,xy,del(20)(q12)[2]/46,xy[18]  
1                                46,xx  
2   46,xy,t(3;3)(q25;q27)[8]/46,xy[12]  
3    46,xy,del(3)(q26q27)[15]/46,xy[5]  
4  46,xx,t(3;9)(p13;q22)[10]/46,xx[10]  

Aperçu des données moléculaires:
        ID CHR        START          END                REF ALT    GENE  \
0  P100000  11  119149248.0  119149248.0                  G   A     CBL   
1  P100000   5  131822301.0  131822301.0       

### Prétraitement des données cliniques

Imputation des valeurs manquantes par la médiane pour les variables continues et transformations logarithmiques pour réduire l'asymétrie.

In [2]:
# Imputation des manquants par la médiane pour les variables continues
num_cols = ['BM_BLAST','WBC','ANC','MONOCYTES','HB','PLT']
for c in num_cols:
    clin[c] = pd.to_numeric(clin[c], errors='coerce')
    clin[c].fillna(clin[c].median(), inplace=True)

# Transformation (log) pour WBC et ANC pour réduire l'asymétrie
clin['WBC'] = np.log1p(clin['WBC'])
clin['ANC'] = np.log1p(clin['ANC'])

print("Variables numériques après imputation et transformation:")
print(clin[num_cols].describe())

Variables numériques après imputation et transformation:
          BM_BLAST          WBC          ANC    MONOCYTES           HB  \
count  3323.000000  3323.000000  3323.000000  3323.000000  3323.000000   
mean      5.884713     1.721027     1.169592     0.849908     9.887142   
std       7.508283     0.614101     0.635635     2.423767     2.007378   
min       0.000000     0.182322     0.000000     0.000000     4.000000   
25%       1.300000     1.335001     0.693147     0.200000     8.600000   
50%       3.000000     1.629241     1.098612     0.370000     9.700000   
75%       8.000000     1.989925     1.508512     0.610000    11.100000   
max      91.000000     5.046002     4.706101    44.200000    16.600000   

               PLT  
count  3323.000000  
mean    165.405185  
std     146.898251  
min       2.000000  
25%      67.000000  
50%     123.000000  
75%     223.500000  
max    1451.000000  


C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\1288892166.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clin[c].fillna(clin[c].median(), inplace=True)
C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\1288892166.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

### Extraction des caractéristiques cytogénétiques

Nous extrayons des caractéristiques cytogénétiques simples : indicateur de caryotype normal (si la chaîne commence par « 46,xx » ou « 46,xy » sans anomalies déclarées), et la présence d'anomalies connues (monosomie 7, gain de chromosome 8). Ces nouvelles variables binaires sont ajoutées aux données cliniques.

In [3]:
import re

# Fonction pour détecter un caryotype normal (46,xx ou 46,xy pur)
def is_normal_karyotype(s):
    if pd.isna(s): 
        return False
    return bool(re.match(r'46,(xx|xy)(\[|$)', s.strip()))
    
# Extraction des caractéristiques cytogénétiques
clin['cyto_normal'] = clin['CYTOGENETICS'].apply(is_normal_karyotype).astype(int)
clin['cyto_mono7'] = clin['CYTOGENETICS'].fillna('').str.contains('-7').astype(int)
clin['cyto_gain8'] = clin['CYTOGENETICS'].fillna('').str.contains('\+8').astype(int)

print("Caractéristiques cytogénétiques créées:")
print(f"Caryotype normal: {clin['cyto_normal'].sum()} patients")
print(f"Monosomie 7: {clin['cyto_mono7'].sum()} patients")
print(f"Gain chromosome 8: {clin['cyto_gain8'].sum()} patients")

Caractéristiques cytogénétiques créées:
Caryotype normal: 1693 patients
Monosomie 7: 169 patients
Gain chromosome 8: 230 patients


<>:12: SyntaxWarning: invalid escape sequence '\+'
<>:12: SyntaxWarning: invalid escape sequence '\+'
C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\20959312.py:12: SyntaxWarning: invalid escape sequence '\+'
  clin['cyto_gain8'] = clin['CYTOGENETICS'].fillna('').str.contains('\+8').astype(int)


### Encodage du centre de traitement

Encodage one-hot du centre de traitement pour capturer les effets de centres différents.

In [4]:
# One-hot encoding du centre de traitement
from sklearn.preprocessing import OneHotEncoder

ctr_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ctr_feat = ctr_ohe.fit_transform(clin[['CENTER']])
ctr_df = pd.DataFrame(ctr_feat, columns=ctr_ohe.get_feature_names_out(['CENTER']))

# Concaténation avec les données cliniques
clin = pd.concat([clin.reset_index(drop=True), ctr_df], axis=1)
clin.drop(['CENTER','CYTOGENETICS'], axis=1, inplace=True)

print(f"Nombre de centres encodés: {ctr_feat.shape[1]}")
print(f"Forme finale des données cliniques: {clin.shape}")

Nombre de centres encodés: 23
Forme finale des données cliniques: (3323, 33)


## 2. Ingénierie des variables moléculaires

Chaque patient possède plusieurs mutations listées dans le fichier moléculaire. Nous agrégeons cette information par patient (ID) pour extraire des variables globales :
- **Nmut** : nombre total de mutations
- **Nstrong** : nombre de mutations « à fort impact » (frameshift, stop-gain, splice-site)
- **VAF_mean** et **VAF_max** : statistiques sur la Fraction Allélique (VAF)
- **Indicateurs de gènes** : présence de mutations dans les gènes fréquemment altérés

In [5]:
# Calcul des agrégats par patient
mol['VAF'] = pd.to_numeric(mol['VAF'], errors='coerce')

# Nombre total de mutations par patient
Nmut = mol.groupby('ID').size().rename('Nmut')

# Compter les mutations à fort impact
strong_impacts = ['stop_gained','frameshift_variant','splice_site_variant']
mol['strong'] = mol['EFFECT'].isin(strong_impacts).astype(int)
Nstrong = mol.groupby('ID')['strong'].sum().rename('Nstrong')

# Statistiques de VAF
VAF_mean = mol.groupby('ID')['VAF'].mean().rename('VAF_mean')
VAF_max = mol.groupby('ID')['VAF'].max().rename('VAF_max')

print("Agrégats moléculaires calculés:")
print(f"Nmut - Mutations totales par patient: {Nmut.describe()}")
print(f"Nstrong - Mutations à fort impact: {Nstrong.describe()}")
print(f"VAF_mean - VAF moyen: {VAF_mean.describe()}")
print(f"VAF_max - VAF maximum: {VAF_max.describe()}")

Agrégats moléculaires calculés:
Nmut - Mutations totales par patient: count    3026.000000
mean        3.613681
std         2.220222
min         1.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        17.000000
Name: Nmut, dtype: float64
Nstrong - Mutations à fort impact: count    3026.000000
mean        1.672835
std         1.571451
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        12.000000
Name: Nstrong, dtype: float64
VAF_mean - VAF moyen: count    3024.000000
mean        0.307040
std         0.159688
min         0.020000
25%         0.192500
50%         0.297000
75%         0.410775
max         0.989000
Name: VAF_mean, dtype: float64
VAF_max - VAF maximum: count    3024.000000
mean        0.445481
std         0.229083
min         0.020000
25%         0.305900
50%         0.442000
75%         0.509250
max         0.999000
Name: VAF_max, dtype: float64


In [6]:
# Indicateurs de présence pour les gènes clés (top 10 gènes les plus mutants)
top_genes = ['TET2','ASXL1','SF3B1','DNMT3A','RUNX1','SRSF2','TP53','STAG2','U2AF1','EZH2']

for gene in top_genes:
    mol[gene] = (mol['GENE'] == gene).astype(int)

gene_features = mol.groupby('ID')[top_genes].max()

print("Indicateurs de gènes créés:")
for gene in top_genes:
    count = gene_features[gene].sum()
    print(f"{gene}: {count} patients avec mutation")

# Fusion de toutes les variables moléculaires
mol_feat = pd.concat([Nmut, Nstrong, VAF_mean, VAF_max, gene_features], axis=1).fillna(0)

print(f"\nForme des caractéristiques moléculaires: {mol_feat.shape}")

Indicateurs de gènes créés:
TET2: 1033 patients avec mutation
ASXL1: 893 patients avec mutation
SF3B1: 739 patients avec mutation
DNMT3A: 534 patients avec mutation
RUNX1: 462 patients avec mutation
SRSF2: 571 patients avec mutation
TP53: 379 patients avec mutation
STAG2: 301 patients avec mutation
U2AF1: 285 patients avec mutation
EZH2: 216 patients avec mutation

Forme des caractéristiques moléculaires: (3026, 14)


### Fusion des données cliniques et moléculaires

Jointure des données cliniques prétraitées avec les caractéristiques moléculaires agrégées pour former la matrice finale d'entraînement.

In [7]:
# Préparation des données pour la jointure (ID en index)
X_clin = clin.set_index('ID')
X_mol = mol_feat

# Jointure des données cliniques et moléculaires
X_all = X_clin.join(X_mol, how='left').fillna(0)

# Préparation de la cible d'entraînement (enlever les cas sans données d'OS)
y = target.dropna().set_index('ID')
X_all = X_all.loc[y.index]
y_years = y['OS_YEARS'].values

print(f"Matrice finale d'entraînement: {X_all.shape}")
print(f"Variable cible (OS_YEARS): {len(y_years)} échantillons")
print(f"Statistiques de survie: min={y_years.min():.2f}, max={y_years.max():.2f}, moyenne={y_years.mean():.2f}")

Matrice finale d'entraînement: (3173, 46)
Variable cible (OS_YEARS): 3173 échantillons
Statistiques de survie: min=0.00, max=22.04, moyenne=2.48


## 3. Modèles prédictifs

Nous entraînons plusieurs modèles de survie pour capter différents aspects des données :

1. **XGBoost** : modèle de boosting pour capturer les interactions complexes
2. **Random Forest** : modèle basé sur les arbres, robuste et stable
3. **Stacking** : méta-modèle combinant les prédictions des modèles de base

### Entraînement des modèles de base

Des modèles basés sur les arbres (Random Forest) et le boosting (XGBoost) pour prédire le temps de survie.

In [8]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Modèle XGBoost (régression sur OS_YEARS)
print("Entraînement du modèle XGBoost...")
xgb_model = XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=200, 
    random_state=0,
    learning_rate=0.1,
    max_depth=6
)
xgb_model.fit(X_all, y_years)

# Forêt aléatoire (RandomForestRegressor) sur OS_YEARS
print("Entraînement du modèle Random Forest...")
rf_model = RandomForestRegressor(
    n_estimators=200, 
    random_state=0,
    max_depth=10,
    min_samples_split=5
)
rf_model.fit(X_all, y_years)

# Évaluation des modèles par validation croisée
xgb_scores = cross_val_score(xgb_model, X_all, y_years, cv=5, scoring='neg_mean_squared_error')
rf_scores = cross_val_score(rf_model, X_all, y_years, cv=5, scoring='neg_mean_squared_error')

print(f"\nPerformances (CV MSE):")
print(f"XGBoost: {-xgb_scores.mean():.4f} (+/- {xgb_scores.std() * 2:.4f})")
print(f"Random Forest: {-rf_scores.mean():.4f} (+/- {rf_scores.std() * 2:.4f})")

print("\nModèles de base entraînés avec succès!")

Entraînement du modèle XGBoost...
Entraînement du modèle Random Forest...

Performances (CV MSE):
XGBoost: 7.3744 (+/- 6.0230)
Random Forest: 7.0060 (+/- 5.2441)

Modèles de base entraînés avec succès!


### Stacking (Empilement) des modèles

Le principe du stacking est d'entraîner plusieurs modèles de base différents et d'utiliser leurs prédictions comme entrées d'un méta-modèle qui apprendra à les combiner optimalement.

In [9]:
from sklearn.linear_model import LinearRegression

# Prédictions des modèles de base sur l'ensemble d'entraînement
print("Génération des prédictions pour le stacking...")
pred_xgb = xgb_model.predict(X_all)
pred_rf = rf_model.predict(X_all)

# Création de la matrice d'empilage (stacking matrix)
stack_X = np.vstack([pred_xgb, pred_rf]).T

print(f"Matrice de stacking: {stack_X.shape}")
print(f"Corrélation entre les prédictions: {np.corrcoef(pred_xgb, pred_rf)[0,1]:.4f}")

# Méta-modèle linéaire sur les prédictions empilées
print("Entraînement du méta-modèle...")
meta_model = LinearRegression().fit(stack_X, y_years)

# Prédiction finale avec le modèle stacké
final_pred = meta_model.predict(stack_X)
final_mse = mean_squared_error(y_years, final_pred)

print(f"\nPerformances du modèle stacké:")
print(f"MSE: {final_mse:.4f}")
print(f"RMSE: {np.sqrt(final_mse):.4f}")

# Coefficients du méta-modèle
print(f"\nCoefficients du méta-modèle:")
print(f"XGBoost: {meta_model.coef_[0]:.4f}")
print(f"Random Forest: {meta_model.coef_[1]:.4f}")
print(f"Intercept: {meta_model.intercept_:.4f}")

Génération des prédictions pour le stacking...
Matrice de stacking: (3173, 2)
Corrélation entre les prédictions: 0.9242
Entraînement du méta-modèle...

Performances du modèle stacké:
MSE: 0.7754
RMSE: 0.8806

Coefficients du méta-modèle:
XGBoost: 1.4421
Random Forest: -0.2321
Intercept: -0.5123


## 4. Prédictions finales et soumission

Application du pipeline complet au jeu de test pour générer les scores de risque. Le score de risque est défini comme l'opposé du temps de survie prédit (plus le score est élevé, plus le risque de décès est grand).

### Préparation des données de test

Application des mêmes étapes de prétraitement aux données de test.

In [10]:
# Chargement des données de test
print("Chargement des données de test...")
clin_test = pd.read_csv('X_test/clinical_test.csv')
mol_test = pd.read_csv('X_test/molecular_test.csv')

print(f"Données cliniques test: {clin_test.shape}")
print(f"Données moléculaires test: {mol_test.shape}")

# Application du même prétraitement aux données cliniques de test
print("Prétraitement des données cliniques de test...")

# Imputation par la médiane (utiliser les mêmes valeurs que l'entraînement)
for c in num_cols:
    clin_test[c] = pd.to_numeric(clin_test[c], errors='coerce')
    # Utiliser la médiane de l'entraînement pour l'imputation
    train_median = clin[c].median() if c in ['BM_BLAST','MONOCYTES','HB','PLT'] else clin[c].median()
    clin_test[c].fillna(train_median, inplace=True)

# Transformations logarithmiques
clin_test['WBC'] = np.log1p(clin_test['WBC'])
clin_test['ANC'] = np.log1p(clin_test['ANC'])

# Caractéristiques cytogénétiques
clin_test['cyto_normal'] = clin_test['CYTOGENETICS'].apply(is_normal_karyotype).astype(int)
clin_test['cyto_mono7'] = clin_test['CYTOGENETICS'].fillna('').str.contains('-7').astype(int)
clin_test['cyto_gain8'] = clin_test['CYTOGENETICS'].fillna('').str.contains('\+8').astype(int)

# Encodage des centres (utiliser le même encodeur)
ctr_feat_test = ctr_ohe.transform(clin_test[['CENTER']])
ctr_df_test = pd.DataFrame(ctr_feat_test, columns=ctr_ohe.get_feature_names_out(['CENTER']))
clin_test = pd.concat([clin_test.reset_index(drop=True), ctr_df_test], axis=1)
clin_test.drop(['CENTER','CYTOGENETICS'], axis=1, inplace=True)

print("Prétraitement des données cliniques de test terminé.")

Chargement des données de test...
Données cliniques test: (1193, 9)
Données moléculaires test: (3089, 11)
Prétraitement des données cliniques de test...
Prétraitement des données cliniques de test terminé.


<>:26: SyntaxWarning: invalid escape sequence '\+'
<>:26: SyntaxWarning: invalid escape sequence '\+'
C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\4286984736.py:26: SyntaxWarning: invalid escape sequence '\+'
  clin_test['cyto_gain8'] = clin_test['CYTOGENETICS'].fillna('').str.contains('\+8').astype(int)
C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\4286984736.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clin_test[c].fillna(train_median, inplace=True)
C:\Users\uthar\AppData\Local\Temp\ipykernel_19288\428698473

In [11]:
# Prétraitement des données moléculaires de test
print("Prétraitement des données moléculaires de test...")

# Calcul des agrégats pour le test
mol_test['VAF'] = pd.to_numeric(mol_test['VAF'], errors='coerce')
Nmut_test = mol_test.groupby('ID').size().rename('Nmut')

# Mutations à fort impact
mol_test['strong'] = mol_test['EFFECT'].isin(strong_impacts).astype(int)
Nstrong_test = mol_test.groupby('ID')['strong'].sum().rename('Nstrong')

# Statistiques VAF
VAF_mean_test = mol_test.groupby('ID')['VAF'].mean().rename('VAF_mean')
VAF_max_test = mol_test.groupby('ID')['VAF'].max().rename('VAF_max')

# Indicateurs de gènes
for gene in top_genes:
    mol_test[gene] = (mol_test['GENE'] == gene).astype(int)
gene_features_test = mol_test.groupby('ID')[top_genes].max()

# Fusion des caractéristiques moléculaires
mol_feat_test = pd.concat([Nmut_test, Nstrong_test, VAF_mean_test, VAF_max_test, gene_features_test], axis=1).fillna(0)

# Jointure finale des données de test
X_clin_test = clin_test.set_index('ID')
X_test = X_clin_test.join(mol_feat_test, how='left').fillna(0)

print(f"Matrice finale de test: {X_test.shape}")

# S'assurer que les colonnes correspondent à celles de l'entraînement
missing_cols = set(X_all.columns) - set(X_test.columns)
extra_cols = set(X_test.columns) - set(X_all.columns)

if missing_cols:
    print(f"Colonnes manquantes dans le test: {missing_cols}")
    for col in missing_cols:
        X_test[col] = 0

if extra_cols:
    print(f"Colonnes supplémentaires dans le test: {extra_cols}")
    X_test = X_test.drop(columns=list(extra_cols))

# Réordonner les colonnes pour qu'elles correspondent
X_test = X_test[X_all.columns]

print(f"Matrice de test finale: {X_test.shape}")
print("Prétraitement des données de test terminé.")

Prétraitement des données moléculaires de test...
Matrice finale de test: (1193, 46)
Matrice de test finale: (1193, 46)
Prétraitement des données de test terminé.


### Génération des prédictions finales

Application du modèle stacké aux données de test pour générer les scores de risque.

In [12]:
# Prédictions des modèles de base sur les données de test
print("Génération des prédictions finales...")

pred_test_xgb = xgb_model.predict(X_test)
pred_test_rf = rf_model.predict(X_test)

# Création de la matrice de stacking pour le test
stack_test = np.vstack([pred_test_xgb, pred_test_rf]).T

# Prédiction finale avec le méta-modèle
pred_test_years = meta_model.predict(stack_test)

# Conversion en scores de risque (plus grand = plus de risque)
risk_score = -pred_test_years

print(f"Prédictions générées pour {len(risk_score)} patients de test")
print(f"Statistiques des scores de risque:")
print(f"  Min: {risk_score.min():.4f}")
print(f"  Max: {risk_score.max():.4f}")
print(f"  Moyenne: {risk_score.mean():.4f}")
print(f"  Médiane: {np.median(risk_score):.4f}")

# Génération du fichier de soumission
submission = pd.DataFrame({
    'ID': X_test.index, 
    'risk_score': risk_score
})

# Sauvegarde du fichier de soumission
submission_filename = 'y_test.csv'
submission.to_csv(submission_filename, index=False)

print(f"\nFichier de soumission généré: {submission_filename}")
print(f"Nombre de prédictions: {len(submission)}")
print("\nAperçu du fichier de soumission:")
print(submission.head(10))

Génération des prédictions finales...
Prédictions générées pour 1193 patients de test
Statistiques des scores de risque:
  Min: -8.5372
  Max: 1.6591
  Moyenne: -1.4857
  Médiane: -1.2336

Fichier de soumission généré: y_test.csv
Nombre de prédictions: 1193

Aperçu du fichier de soumission:
      ID  risk_score
0   KYW1   -0.621691
1   KYW2   -0.777121
2   KYW3   -1.748210
3   KYW4   -0.186051
4   KYW5   -0.962737
5   KYW6   -0.329525
6   KYW7   -0.850392
7   KYW8   -1.636858
8   KYW9   -1.709925
9  KYW10   -2.288068


## 5. Résumé et conclusion

### Approche utilisée

Nous avons construit un modèle de survie combinant plusieurs techniques d'apprentissage automatique :

1. **Prétraitement des données** :
   - Imputation des valeurs manquantes par la médiane
   - Transformations logarithmiques pour réduire l'asymétrie
   - Extraction de caractéristiques cytogénétiques binaires
   - Encodage one-hot des centres de traitement
   - Agrégation des données moléculaires par patient

2. **Modèles de base** :
   - **XGBoost** : pour capturer les interactions complexes et non-linéaires
   - **Random Forest** : pour la robustesse et la stabilité des prédictions

3. **Stacking (Empilement)** :
   - Méta-modèle linéaire combinant les prédictions des modèles de base
   - Optimise le score de concordance en fusionnant les perspectives complémentaires

### Avantages de cette approche

- **Robustesse** : Les Random Forests sont réputées robustes pour la survie
- **Flexibilité** : XGBoost capture les interactions complexes
- **Amélioration** : Le stacking combine les forces de chaque modèle
- **Généralisation** : L'ensemble réduit le sur-apprentissage

### Sources et références

- **Random Survival Forests** : Extension des forêts aléatoires à la censure (scikit-survival)
- **Stacking** : Technique de méta-apprentissage pour combiner plusieurs modèles
- **XGBoost** : Algorithme de boosting efficace pour les problèmes de régression

Cette approche d'ensemble permet de capturer à la fois les interactions complexes et d'augmenter la robustesse du pronostic pour optimiser le score de concordance final.